[![Binder](https://mybinder.org/badge.svg)](https://mybinder.org:/v2/gh/jinjagit/Cryptography/master?filepath=StreamCipher.ipynb)
<h1>Stream Cipher</h1>
<br>
Note: many diagrams and some other items will not display in static views (like GitHub).
<br>

Notes on Lecture 3, [Introduction to Cryptography](https://www.youtube.com/channel/UC1usFRN4LCMcfIV7UjHNuQg/videos), by Christof Paar, and Chapter 2, [Understanding Crpytography](https://www.amazon.com/Understanding-Cryptography-Christof-Paar-ebook/dp/B00HWUO98A), by Paar & Pletzl:

<center><img src=diagrams/CryptoTree.svg / ></center>

Note: GSM is used by all mobile phones, was developed in mid 1980s, and was first large-scale application of encrytpion. GSM uses a stream cipher to encrypt digital voice (audio) signals before transmitting to base towers.

**Definition:** A stream cipher encrypts bits individually (as against a block cipher, which encodes blocks of bits), **by adding a bit from a <i>key stream</i> to a plaintext bit.**

Usually, stream ciphers use very simple encrytption and decryption methods.

Each bit $x_i$ is encrypted by adding a secret key stream bit $s_i$, modulo 2.

<center><img src=diagrams/asynchStream.svg / ></center>

Stream ciphers tend to be small and fast and are, therefore, relevant to applications that run in environments with relatively limited resources. In general, however, block ciphers are still used more frequenly for encrytping computer communications. Neither kind can claim better overall efficiency (of software = cycles; or hardware = fewer gates/smaller chip area).

**Encryption and Decryption:**

<i>The plaintext, ciphertext and key consist of individual bits;</i>

$x_i, y_i, s_i \in\{1, 0\}$

**Encryption:** $y_i = e_{s_i}(x_i)\equiv x_i + s_i\mod{2}$<br></br>
**Decryption:** $x_i = d_{s_i}(y_i)\equiv y_i + s_i\mod{2}$

  1. Encryption and decryption are same functions!
  2. Why can we use a simple modulo 2 addition as encrytpion?
  3. What is the nature of the stream bits $s_i$?